In [ ]:
import turtle
import numpy as np
from math import atan2
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Constants for the Lorenz-like system
rho = 43.5
sigma = 10.0
beta = 8.0 / 3.0
d0 = 19.0 / 3.0

# Initial scale and dynamic range
initial_scale = 10
scale = 10
offset_x, offset_y = 0, 0  # To center the plot dynamically

# Define the system of equations
def f(state, t):
    x, y, z, y1, z1 = state
    return (
        sigma * (y - x),
        x * (rho - z) - y,
        x * y - x * y1 - beta * z,
        x * z - 2 * x * z1 - d0 * y1,
        2 * x * y1 - 4 * beta * z1
    )

# Initial conditions and time range
state0 = [1.0, 1.0, 1.0, 1.0, 1.0]
t = np.arange(0.0, 200.0, 0.01)  # Increased time range

# Solve the differential equations
states = odeint(f, state0, t)

# Get bounds for dynamic scaling
x_vals, z_vals = states[:, 0], states[:, 2]
offset_x, offset_y = np.mean(x_vals), np.mean(z_vals)
max_range = max(np.max(np.abs(x_vals)), np.max(np.abs(z_vals)))
scale = 400 / max_range

# Initialize Turtle
turtle.screensize(canvwidth=7680, canvheight=4800, bg="black")
plotter = turtle.Turtle()
plotter.speed("fastest")
plotter.pensize(1)
plotter.radians()
plotter.pendown()

# Create a color gradient
colors = plt.cm.plasma(np.linspace(0, 1, len(states)))

# Function to draw using turtle dynamically
def draw_lorenz():
    global states
    while True:
        for i, (x, y, z, y1, z1) in enumerate(states):
            plotter.setpos((x - offset_x) * scale, (z - offset_y) * scale)
            plotter.setheading(atan2(z1, y1))
            plotter.pencolor(colors[i][0], colors[i][1], colors[i][2])
        # Recompute states dynamically
        final_state = states[-1]
        t_next = np.arange(t[-1] + 0.01, t[-1] + 200.0, 0.01)
        states = odeint(f, final_state, t_next)

# Optional: Matplotlib animation for visualization
def matplotlib_animation():
    fig, ax = plt.subplots()
    ax.set_xlim(-50, 50)
    ax.set_ylim(-50, 50)
    line, = ax.plot([], [], lw=2)

    def init():
        line.set_data([], [])
        return line,

    def update(frame):
        line.set_data(states[:frame, 0], states[:frame, 2])
        return line,

    ani = FuncAnimation(fig, update, frames=len(states), init_func=init, blit=True)
    plt.show()

# Run the options
print("Choose visualization mode:\n1: Turtle Continuous Drawing\n2: Matplotlib Animation")
choice = input("Enter choice (1 or 2): ")
if choice == "1":
    draw_lorenz()
elif choice == "2":
    matplotlib_animation()
else:
    print("Invalid choice.")

# Keep the window open until closed
turtle.done()


Choose visualization mode:
1: Turtle Continuous Drawing
2: Matplotlib Animation
